CONCATENING TEXT DATASETS, DON'T HAVE TO RUN THIS SECTION ANYMORE

In [76]:
#DONT RUN THIS ANYMORE

from datasets import load_dataset, Dataset
from convokit import Corpus, download

ds_ag_news = load_dataset("fancyzhx/ag_news") #news, formal
ds_sms = load_dataset("ucirvine/sms_spam") #should only use ham labels, text messages, informal
ds_edu = load_dataset("Locutusque/UltraTextbooks") #synthetic and human written textbooks..... acdademic / formal
#https://huggingface.co/datasets/fabiochiu/medium-articles
ds_medium = load_dataset('csv', data_files='/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/medium_articles.csv') #medium articles, academic but also likely less formal ? community contributions
#https://huggingface.co/datasets/kariatouk/enron_curated_labeled/tree/main
ds_enron= load_dataset('csv', data_files='/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/enron_labeled_curated.csv') #enron, very formal business emails probably?


ds_podcast_news = load_dataset("Whispering-GPT/lex-fridman-podcast")
ds_telephone_convos = Corpus(filename=download("switchboard-corpus"))

data = ds_telephone_convos.get_utterances_dataframe() 
data_list = data.to_dict(orient="records")
ds_telephone_convos = Dataset.from_list(data_list)

ds_ted_talk_edu = load_dataset('csv', data_files="/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/ted_talks_en.csv")
ds_un_debates = load_dataset('csv', data_files="/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/un-general-debates.csv")
#city council meetings instead https://huggingface.co/datasets/lytang/MeetingBank-transcript?row=8&library=datasets
ds_business_meetings = load_dataset("lytang/MeetingBank-transcript")
#this cell ran in 21 minutes (probably don't have to load the whole dataset?)



Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/91 [00:00<?, ?it/s]

Dataset already exists at /Users/rheamadhogarhia/.convokit/saved-corpora/switchboard-corpus


In [79]:
ds_telephone_convos[0:2]

{'timestamp': [None, None],
 'text': ['Okay.  /{D So, }', '[ [ I guess, +'],
 'speaker': ['1519', '1632'],
 'reply_to': [None, '4325-0'],
 'conversation_id': ['4325-0', '4325-0'],
 'meta.tag': [[['Okay.  /', 'o'], ['{D So, }', 'qw']],
  [['[ [ I guess, +', 'qy^d']]],
 'vectors': [[], []]}

In [91]:
#TRAIN SETS, choosing 500 samples frome each written text set
num_samples = 500

ds_ag_news_sub = ds_ag_news["train"].shuffle(seed=42).select(range(num_samples))
#filtering out the spam label - only getting legitimate texts
ds_sms_filtered = ds_sms["train"].filter(lambda example: example['label'] == 0)
ds_sms_sub = ds_sms_filtered.shuffle(seed=42).select(range(num_samples))
ds_edu_sub = ds_edu["train"].shuffle(seed=42).select(range(num_samples))
ds_medium_sub = ds_medium["train"].shuffle(seed=42).select(range(num_samples))
ds_enron_sub = ds_enron["train"].shuffle(seed=42).select(range(num_samples))


ds_podcast_news_sub = ds_podcast_news["train"].shuffle(seed=42).select(range(100))#podcasts are long so only gonna use 100 samples
ds_telephone_sub = ds_telephone_convos.shuffle(seed=42).select(range(num_samples))
ds_ted_talk_edu_sub = ds_ted_talk_edu["train"].shuffle(seed=42).select(range(num_samples))
ds_business_meetings_sub = ds_business_meetings["train"].shuffle(seed=42).select(range(num_samples))
ds_un_debates_sub = ds_un_debates["train"].shuffle(seed=42).select(range(num_samples))


In [85]:
ds_un_debates_sub[0:2]

{'session': [58, 65],
 'year': [2003, 2010],
 'country': ['ISR', 'TGO'],
 'text': ['\ufeffI would like to congratulate\nHis Excellency, the Foreign Minister of Saint Lucia,\nupon his assumption of the Presidency of the General\nAssembly, and wish him much success.\nUntil just one month ago, every person in this hall\nand every Member of this Organization joined us in the\nhope that the Middle East peace process might finally\nbe back on track, and that a resolution of the Israeli-\nPalestinian conflict might be on the horizon. The\nestablishment of a new Palestinian Government\npromised an end to terror and a new beginning.\nThis glimmer of hope was darkened on 19 August\nby the extremists who blew up a bus full of Jewish\nfamilies on their way home from prayers at the Western\nWall, the holiest site in the Jewish religion. Twenty-\nthree people, young and old, mothers and babies in\ntheir cradles, were slaughtered in that attack. That\nattack was carried out by Hamas, a terrorist\norg

In [94]:
#combine
from datasets import Dataset, concatenate_datasets
import random

ag_news_dataset = Dataset.from_dict({"text": ds_ag_news_sub["text"], 
                                     "source": ["ag_news"] * len(ds_ag_news_sub),
                                     "formality": ["formal"] * len(ds_ag_news_sub)})
medium_dataset = Dataset.from_dict({"text": ds_medium_sub["text"], 
                                    "source": ["medium"] * len(ds_medium_sub),
                                    "formality": ["formal"] * len(ds_medium_sub)})
edu_dataset = Dataset.from_dict({"text": ds_edu_sub["text"], 
                                 "source": ["textbooks"] * len(ds_edu_sub),
                                 "formality": ["educational"] * len(ds_edu_sub)})
sms_dataset = Dataset.from_dict({"text": ds_sms_sub["sms"], 
                                 "source": ["sms"] * len(ds_sms_sub), 
                                 "formality": ["informal"] * len(ds_sms_sub)})
enron_dataset = Dataset.from_dict({"text": ds_enron_sub["email"], 
                                   "source": ["enron_emails"] * len(ds_enron_sub), 
                                   "formality": ["formal"] * len(ds_enron_sub)})


combined_written_dataset = concatenate_datasets([ag_news_dataset, medium_dataset, edu_dataset, sms_dataset, enron_dataset])
combined_written_dataset = combined_written_dataset.add_column("type", ["written"] * len(combined_written_dataset))



podcast_news_dataset = Dataset.from_dict({"text": ds_podcast_news_sub["text"], 
                                     "source": ["podcasts"] * len(ds_podcast_news_sub), 
                                     "formality": ["informal"] * len(ds_podcast_news_sub)})
un_debates_dataset = Dataset.from_dict({"text": ds_un_debates_sub["text"], 
                                   "source": ["un_debates"] * len(ds_un_debates_sub),
                                   "formality": ["formal"] * len(ds_un_debates_sub)})
ted_talk_edu_dataset = Dataset.from_dict({"text": ds_ted_talk_edu_sub["transcript"], 
                                 "source": ["ted_talks"] * len(ds_ted_talk_edu_sub),
                                 "formality": ["educational"] * len(ds_ted_talk_edu_sub)})
telephone_dataset = Dataset.from_dict({"text": ds_telephone_sub["text"], 
                                    "source": ["telephone_convos"] * len(ds_telephone_sub), 
                                    "formality": ["informal"] * len(ds_telephone_sub)})
business_meetings_dataset = Dataset.from_dict({"text": ds_business_meetings_sub["source"], 
                                 "source": ["business_meetings"] * len(ds_business_meetings_sub),
                                 "formality": ["formal"] * len(ds_business_meetings_sub)})

combined_spoken_dataset = concatenate_datasets([podcast_news_dataset, un_debates_dataset, ted_talk_edu_dataset, telephone_dataset, business_meetings_dataset])
combined_spoken_dataset = combined_spoken_dataset.add_column("type", ["spoken"] * len(combined_spoken_dataset))

full_dataset = concatenate_datasets([combined_spoken_dataset, combined_written_dataset])

full_dataset = full_dataset.shuffle(seed=42)
full_dataset.to_csv("spoken_written_full_dataset.csv")

print(full_dataset[:5])


Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

{'text': ['Ukraine appears set for new poll It looks likely that Ukraine will hold a new election as it awaits a supreme court verdict on electoral fraud.', 'Mozambique Prepares for Elections to Replace Chissano Mozambique holds elections on Wednesday and Thursday to choose a successor to President Joaquim Chissano, who ruled for 18 years and led the southern African country to peace after a long civil war that claimed up to a million lives.', "The year that has elapsed since the forty-\nseventh session of the General Assembly was noteworthy for\nthe further growth in the authority and the effectiveness of\nthe United Nations. It testifies, in our view, to the deep\nunderstanding that the members of the world community\nhave about the need for close, constructive cooperation in\nthe name of a better future. As I look around this Hall, in\nwhich, for the first time, a Head of State of a sovereign\nTajikistan is speaking, the words of the great poet Saadi\ncome to my mind:\n'The sons of 

In [95]:
#Load in new dataset 
full_dataset = load_dataset('csv', data_files='spoken_written_full_dataset.csv') #medium articles, academic but also likely less formal ? community contributions

#preprocessing to do
#lowercase
#get rid of links
#get rid of code?
# get rid of maybe = signs and equations
#speaker 1 etc? maybe we need to preprocess each source differenlty


Generating train split: 0 examples [00:00, ? examples/s]